In [3]:
import json
import pandas as pd
from utils import prep_ans
import numpy as np
from vqa_question_type_processing import question_type_processing as vqa_question_type_processing
from simpsons_question_type_processing import question_type_processing as simpsons_question_type_processing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from io import BytesIO
from PIL import Image
import numpy as np
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

In [4]:
vqav2_anno_paths = ["/home/ngoc/data/vqav2/new_vqa2_val_annotation.json", "/home/ngoc/data/vqav2/new_vqa2_test_annotation.json"]

In [5]:
vqa_ans_to_idx, idx_to_vqa_ans, question_type_map = json.load(open("/home/ngoc/githubs/chatgpt-hie-vqa/datasets/answer_dicts_vqav2.json", 'r'))

In [6]:
vqa_question_type_dict = {}
with open("/home/ngoc/githubs/Question-Analysis/datasets/train_question_type_gpt_v6_vqav2.json", 'r') as file:
    for line in file:
        question_object = json.loads(line)
        question_str = question_object["question"]
        question_type = question_object["question_type"]
        question_type = vqa_question_type_processing(question_type)
        vqa_question_type_dict[question_str] = question_type
file.close()

In [15]:

vqa_question_paths = ["/home/ngoc/data/vqav2/new_vqa2_val_question.json", "/home/ngoc/data/vqav2/new_vqa2_test_question.json"]
vqa_questions = []
for vqa_question_path in vqa_question_paths:
    with open(vqa_question_path, 'r') as file:
        vqa_questions += json.load(file)
        
        
vqa_processed_questions = {}
for question in vqa_questions:
    # print(question)
    # print(vqa_question_type_dict[question["question"])
    # a
    vqa_processed_questions[question["question_id"]] = vqa_question_type_dict[question["question"]]


In [16]:
vqav2_answers = []
for vqav2_anno_path in vqav2_anno_paths:
    with open(vqav2_anno_path, 'r') as file:
        vqav2_answers += json.load(file)

In [17]:
def proc_ans(ans, ans_to_ix):
    ans_prob_dict = {}
    for ans_ in ans['answers']:
        ans_proc = prep_ans(ans_['answer'])
        if ans_proc not in ans_prob_dict:
            ans_prob_dict[ans_proc] = 1
        else:
            ans_prob_dict[ans_proc] += 1
    answer_scores = {} 
    for ans_ in ans_prob_dict:
        if ans_ in ans_to_ix:
            answer_scores[ans_] = get_score(ans_prob_dict[ans_])
    return answer_scores

def get_score(occur):
    if occur == 0:
        return .0 
    elif occur == 1:
        return round(1/3,2)
    elif occur == 2:
        return round(2/3,2)
    elif occur >= 3:
        return 1

In [18]:
vqav2_processed_answers = {}
for vqav2_answer in  vqav2_answers:
    question_id = vqav2_answer["question_id"]
    vqav2_processed_answers[question_id] = {}
    multiple_choice_answer =  prep_ans(vqav2_answer['multiple_choice_answer'])
    process_answers = proc_ans(vqav2_answer, vqa_ans_to_idx)
    vqav2_processed_answers[question_id]["answer_dict"] = process_answers
    vqav2_processed_answers[question_id]["multiple_choice_answer"] = multiple_choice_answer
    vqav2_processed_answers[question_id]["question_type"] = vqa_processed_questions[question_id]
    

In [19]:
val_result = pd.read_csv("val_predictions.csv")
test_result = pd.read_csv("test_predictions.csv")


In [20]:
def map_answer_scores(question_id, vqa_prediction):
    score_dict = vqav2_processed_answers[question_id]["answer_dict"]
    if vqa_prediction in score_dict.keys():
        return score_dict[vqa_prediction]
    else:
        return 0

In [21]:
# val_result["vqa_prediction"] = val_result["prediction"]
# test_result["vqa_prediction"] = test_result["prediction"]

In [22]:
val_result["vqa_score"] = val_result[["question_id", "vqa_prediction"]].apply(lambda x: map_answer_scores(x["question_id"], x["vqa_prediction"]), axis = 1)
val_result["question_label"] = val_result["question_id"].apply(lambda x: vqav2_processed_answers[x]["question_type"])

In [23]:
val_result["vqa_score"].sum()/len(val_result)

0.5680854397381943

In [24]:
classification_report(val_result["question_prediction"], val_result["question_label"], output_dict=True)["accuracy"]

0.9531626820099567

In [25]:
test_result["vqa_score"] = test_result[["question_id", "vqa_prediction"]].apply(lambda x: map_answer_scores(x["question_id"], x["vqa_prediction"]), axis = 1)
test_result["question_label"] = test_result["question_id"].apply(lambda x: vqav2_processed_answers[x]["question_type"])
test_result["multiple_choice_answer"] = test_result["question_id"].apply(lambda x: vqav2_processed_answers[x]["multiple_choice_answer"])

In [26]:
test_result = test_result[test_result["multiple_choice_answer"].isin(vqa_ans_to_idx.keys())]

In [27]:
test_overall_accuracy  = test_result["vqa_score"].sum()/len(test_result)

In [28]:
sorted_unique_labels = ["yes/no", "object", "color", "number", "location","human", "action", "sport", "other"]

In [29]:
test_overall_accuracy

0.5449592059175229

In [30]:
sub_accurancy = {}
for question_type_value in sorted_unique_labels:
    sub_test_result = test_result[test_result["question_label"] == question_type_value]
    sub_accuracy_i = sub_test_result["vqa_score"].sum()/len(sub_test_result)
    sub_accurancy["test_vqa_accuracy_" + question_type_value] = sub_accuracy_i

In [31]:
sub_accurancy

{'test_vqa_accuracy_yes/no': 0.6705244693021483,
 'test_vqa_accuracy_object': 0.45501823178142997,
 'test_vqa_accuracy_color': 0.45560325810283386,
 'test_vqa_accuracy_number': 0.37053732602277517,
 'test_vqa_accuracy_location': 0.5301192472517235,
 'test_vqa_accuracy_human': 0.4644654498044329,
 'test_vqa_accuracy_action': 0.5544229607250755,
 'test_vqa_accuracy_sport': 0.7490697674418604,
 'test_vqa_accuracy_other': 0.46175619047619043}

In [32]:
run_id = "bi6l2w1x"

In [33]:

wandb.init(project="VQA new", id=run_id, resume="must")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ngocdunghuynh. Use `wandb login --relogin` to force relogin


In [34]:
wandb.log({"test_vqa_accuracy": test_overall_accuracy})

In [35]:
wandb.log(sub_accurancy)

In [36]:
test_question_type_accuracy = classification_report(test_result["question_prediction"], test_result["question_label"], output_dict=True)

In [37]:
new_test_question_type_accuracy = {}
for key in test_question_type_accuracy:
    new_test_question_type_accuracy["question_type_" + key] = test_question_type_accuracy[key]
del(test_question_type_accuracy)

In [38]:
wandb.log(new_test_question_type_accuracy)

In [39]:
y_true = test_result['question_label']
y_pred = test_result['question_prediction']
sorted_unique_labels = ["yes/no", "object", "color", "number", "location","human", "action", "sport", "other"]

conf_matrix = confusion_matrix(y_true, y_pred, labels=sorted_unique_labels)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(20, 9))
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix, annot=True, fmt='d', 
            xticklabels=sorted_unique_labels, yticklabels=sorted_unique_labels, 
            cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', 
            xticklabels=sorted_unique_labels, yticklabels=sorted_unique_labels, 
            cmap='Blues', cbar=False)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Type')
plt.ylabel('True Type')
plt.suptitle(f'Test Confusion Matrix')
buffer = BytesIO()
plt.savefig(buffer, format='png')
buffer.seek(0)
image = Image.open(buffer)
image_array = np.array(image)
wandb.log({"Test Confusion Matrix": wandb.Image(image_array)})
plt.close()
plt.show()

In [40]:
wandb.finish()

wandb: WARNING Ensure read and write access to run files dir: /home/ngoc/githubs/chatgpt-hie-vqa/result_analysis/wandb/run-20240325_152136-bi6l2w1x/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


question_type_accuracy,▁
test_vqa_accuracy,▁
test_vqa_accuracy_action,▁
test_vqa_accuracy_color,▁
test_vqa_accuracy_human,▁
test_vqa_accuracy_location,▁
test_vqa_accuracy_number,▁
test_vqa_accuracy_object,▁
test_vqa_accuracy_other,▁
test_vqa_accuracy_sport,▁
test_vqa_accuracy_yes/no,▁
